In [25]:
# import pandas as pd
!unzip ../nyt_extracted.zip

Archive:  ../nyt_extracted.zip
  inflating: nyt_structured_data.csv.201811032210  
  inflating: nyt_structured_data.txt  


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('nyt_structured_data.txt', sep = '\t', header=None)

In [23]:
data.iloc[0,0]

"1987/03/03/0018316.xml , 'DATA DESIGN LABORATORIES reports earnings for Qtr to Dec 31' , 'LEAD:*3*** COMPANY REPORTS ***3*DATA DESIGN LABORATORIES (NYSE)Qtr to Dec 3119861985Revenue18,316,00016,858,000Net inc25,000251,000Share earns-.046mo rev35,571,00036,756,000Net incb6,187,000b762,000Share earns.94.12' , 'LEAD:*3*** COMPANY REPORTS ***3*DATA DESIGN LABORATORIES (NYSE)Qtr to Dec 3119861985Revenue18,316,00016,858,000Net inc25,000251,000Share earns-.046mo rev35,571,00036,756,000Net incb6,187,000b762,000Share earns.94.12*3*** COMPANY REPORTS ***3*DATA DESIGN LABORATORIES (NYSE)Qtr to Dec 3119861985Revenue18,316,00016,858,000Net inc25,000251,000Share earns-.046mo rev35,571,00036,756,000Net incb6,187,000b762,000Share earns.94.12b-Includes a pre-tax gain of $9,414,000 on the sale of marketable securities, compared with a similar gain of $116,000 for a year earlier.'"

In [22]:
index = 27

print("title is", data.iloc[index,0].split(".xml ,")[1].split("LEAD:")[0])
print("===============================================")

print("abstract is", data.iloc[index,0].split(".xml ,")[1].split("LEAD:")[1])
print("===============================================")

print("body is", data.iloc[index,0].split(".xml ,")[1].split("LEAD:")[2])
print("===============================================")

TR = TextRanker(data.iloc[index,0].split(".xml ,")[1].split("LEAD:")[2])
estimated_summary = TR.run()
print("estimated summary is ", estimated_summary)

title is  '40-JET DEAL EXPECTED AT AMERICAN' , '
abstract is  American Airlines is expected to announce today that it is buying 40 wide-bodied aircraft - 25 from Airbus Industrie and 15 from the Boeing Company - and 80 engines from the General Electric Company to power the aircraft.' , '
body is  American Airlines is expected to announce today that it is buying 40 wide-bodied aircraft - 25 from Airbus Industrie and 15 from the Boeing Company - and 80 engines from the General Electric Company to power the aircraft.American Airlines is expected to announce today that it is buying 40 wide-bodied aircraft - 25 from Airbus Industrie and 15 from the Boeing Company - and 80 engines from the General Electric Company to power the aircraft.Industry sources said the orders would total $2.5 billion.American split the order between the two aircraft manufacturers as part of an effort to get a better price and conditions, but it decided to give the engine order to one manufacturer, the sources said.T

In [3]:
data.head()

,0
0,"1987/03/03/0018316.xml , 'DATA DESIGN LABORATO..."
1,"1987/03/03/0018302.xml , 'MOBIL'S CLEAR PACKAG..."
2,"1987/03/03/0018289.xml , 'Russia Offers to Aid..."
3,"1987/03/03/0018262.xml , 'AFTER 7 DECADES OF C..."
4,"1987/03/03/0018276.xml , 'BOWLING: BOOM SPORT ..."


In [36]:
data.shape

(1855660, 1)

In [33]:
import random

In [19]:

from TextRanker import TextRanker

In [35]:
def summarization(df):
    out_df = []
    for ind in range(len(df)):
        try:
            input_header, input_abstract, input_body = df.iloc[ind,0].split(".xml ,")[1].split("LEAD:")
            TR = TextRanker(input_body)
            estimated_summary = TR.run()
            out_df.append([input_header, input_abstract, estimated_summary])
        except Exception as err:
            continue
    return pd.DataFrame(out_df, columns=["input_header","input_abstract", "estimated_summary"])

In [36]:
test_data = data.sample(100)
out = summarization(test_data)

In [37]:
print("test_data is", test_data)
print("========================")
print("output_data is", out)


test_data is                                                          0
715173   1994/08/12/0705807.xml , 'Abroad at Home; The ...
214689   1989/03/26/0234684.xml , 'CHILDREN'S BOOKS/POE...
872367   1996/09/22/0878726.xml , 'Wild Things From Man...
968088   1997/01/06/0899864.xml , 'Radio Free Asia's Si...
470248   1991/08/04/0465033.xml , 'The World; In the De...
1739557  2006/04/10/1753534.xml , 'The Egg Roll (Again!...
154604   1988/10/10/0186696.xml , 'Redskins Roll Past C...
337132   1990/05/30/0357526.xml , 'American Goalies Wai...
1224606  2000/09/26/1233873.xml , 'The Rural Life; The ...
374506   1990/09/04/0381645.xml , 'Tennis;   Flashily a...
598182   1993/05/12/0608410.xml , 'Fiat, in Scandal, Ad...
238310   1989/02/03/0219944.xml , 'Zonic Corp reports e...
636696   1993/08/09/0627588.xml , 'Jennifer J. Semel, C...
4437     1987/03/07/0019359.xml , 'NEW YORK CITY'S JOBL...
1480977  2003/02/19/1466165.xml , 'World Business Brief...
1818856  2007/03/28/1836214.xml , 'Episcopa

In [44]:
# input_abstract, estimated_summary = out.iloc[0,0
print("input_abstract is ", out.iloc[1]['input_abstract'])
print("=================================================")

print("output_abstract is ", out.iloc[1]['estimated_summary'])

input_abstract = out.iloc[1]['input_abstract']
estimated_summary = out.iloc[1]['estimated_summary']

input_abstract is   Mark Rypien threw three touchdown passes and ran for one score today to lead the Washington Redskins to a 35-17 victory over the error-prone Dallas Cowboys.' , '
output_abstract is  The Cowboys led in the first quarter when Timmy Newsome scored from the 1-yard line, after a Rypien fumble. The Houston backup quarterback Brent Pease, frustrated by three earlier pass interceptions, ran 4 yards for a third quarter touchdown to rally the Oilers ( 4- 2 ). The Kansas City quarterback, Steve DeBerg, led the Chiefs to both of their field goals with three straight pass completions in each drive. Chargers 17SAN DIEGO ( AP )- Bobby Hebert threw two touchdown passes and Morten Andersen kicked three field goals as New Orleans won its fifth straight game. The Raiders' quarterback, Jay Schroeder, had four passes intercepted for the second straight game.


In [42]:
def evaluation(input_abstract, estimated_summary):
    
    input_abstract_words = input_abstract.split(" ")
    estimated_summary_words = estimated_summary.split(" ")
    
    over_lap = []
    for word in input_abstract_words:
        if word in estimated_summary_words:
            over_lap.append(word)
    
    recall_count = 0
    for word in over_lap:
        if word in input_abstract_words:
            recall_count += 1
    
    precision_count = 0
    for word in over_lap:
        if word in estimated_summary_words:
            precision_count += 1
    
    if len(input_abstract_words) >0:
        recall = recall_count / len(input_abstract_words)
    else: 
        recall = 0
        
    if len(estimated_summary_words) >0:
        precision = precision_count / len(estimated_summary_words)
    else:
        precision = 0
        
    print("recall is", recall)
    print("===================")
    print("precision is", precision)
        

In [45]:
evaluation(input_abstract, estimated_summary)

recall is 0.4482758620689655
precision is 0.11711711711711711


In [40]:
# input_data = data[0]
TR = TextRanker(input_body)
# t0 = time.time()
s, w = TR.run()
print("summary:", s)
print("keywords:", w)
# print(time.time() - t0)

summary: If we continue our efforts, there's not going to be a Mafia in 5 to 10 years.''He Giuliani's statements in Providence went significantly beyond those of other Federal officials who have spoken on organized crime. Last April, a Presidential commission on organized crime said the Mafia remains a dominant force in an underworld that makes $ 100 billion a year. Giuliani cautioned that the decline of the Mafia did not mean the end of organized crime. Those prosecutions have disrupted the management of the mafia, he said. Further, prosecutors are pursuing efforts under Federal racketeering laws to seize the assets of the Mafia. The problems facing the Mafia are compounded by the improved status of Italian-Americans, he said.
keywords: {'mafia leaders', 'new members', 'lower manhattan', 'sicilian mafia', 'law school', 'federal jury', 'new york', 'high school', 'mafia', 'providence', 'recent years', 'organized crime', 'mafia figures', 'news conference', 'schools', 'crime'}
